Figure 5 is a sensitivity analysis across various model parameter variations

In [1]:
include(joinpath("..","src","setup.jl"))

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840
┌ Warning: combine(gd) is deprecated, use DataFrame(gd) instead
│   caller = |>(::GroupedDataFrame{DataFrame}, ::typeof(combine)) at operators.jl:813
└ @ Base ./operators.jl:813
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/09PWZ/src/mapping.jl:228


RObject{VecSxp}
$rcalljl_options
$rcalljl_options$height
[1] 360

$rcalljl_options$width
[1] 432




In [2]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ,
          :f_τ_a,:f_τ_m,:s_τ_a,:s_τ_m,:t_τ_a,:t_τ_m,
          :f_W_m_σ,:s_W_m_σ,:t_W_m_σ_t,:t_W_m_σ_ϕ,:t_W_m_σ_N];

In [3]:
param_sets = []
data_sets = []

0-element Array{Any,1}

## Time constant data set

In [4]:
datadir = joinpath("..","data","count_lengths","run_2018-12-06")

params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)
settings = joinpath("..","src","settings.toml")

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


In [5]:
# turns out there are some non-unique parameter values with this; remove those (in the future do it before running the simulation)
paramssel = unique!(params,[fields;:Δf])
dfsel = @where(df,in.(:pindex,Ref(paramssel.pindex)));

paramssel[:old_pindex] = paramssel.pindex;
dfsel[:old_pindex] = dfsel.pindex;

paramssel[:pindex] = 1:size(paramssel,1)
for row in 1:size(dfsel,1)
    i = findfirst(isequal(dfsel.old_pindex[row]),paramssel.old_pindex)
    if i isa Nothing
        error("Can't find old_pindex $(dfsel.old_pindex[row])")
    end
    dfsel.pindex[row] = paramssel.pindex[i]
end
head(dfsel)

delete!(dfsel,:old_pindex);
delete!(paramssel,:old_pindex);

print(maximum(paramssel.pindex))
size(paramssel)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[5]:16
└ @ Core In[5]:16
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `deletecols!(df, cols)` instead.
│   caller = top-level scope at In[5]:17
└ @ Core In[5]:17


1575

┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `deletecols!(df, cols)` instead.
│   caller = top-level scope at In[5]:19
└ @ Core In[5]:19


(1575, 36)

In [6]:
length(unique(dfsel.pindex))

1575

In [7]:
push!(param_sets,paramssel);
push!(data_sets,dfsel);

## Noise magnitude data set

In [8]:
datadir = joinpath(datadir,"..","run_2018-12-14")
params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [9]:
push!(param_sets,params);
push!(data_sets,df);

## Inhibition breadth data set

In [10]:
datadir = joinpath(datadir,"..","run_2019-01-02")
params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [11]:
push!(param_sets,params);
push!(data_sets,df);

## Combine data sets

In [12]:
for i in 2:length(param_sets)
    @show offset = maximum(param_sets[i-1].pindex)
    param_sets[i].pindex .+= offset
    data_sets[i].pindex .+= offset
end
    
df = vcat(data_sets...);
params = vcat(param_sets...);

offset = maximum((param_sets[i - 1]).pindex) = 1575
offset = maximum((param_sets[i - 1]).pindex) = 2250


In [13]:
@assert length(unique(df.pindex)) == size(params,1)

## Summarize data

In [14]:
progress = Progress(length(groupby(params,fields)))
herr = human_error()
df_summary = by(params,fields) do row
    ## next!(progress)
    err = model_error(df,params;(k => row[1,k] for k in fields)...)
    @assert row.Δf == [3,6,12]
    df3 = df[df.pindex .== row[1,:pindex][1],:]
    df6 = df[df.pindex .== row[2,:pindex][1],:]
    df12 = df[df.pindex .== row[3,:pindex][1],:]
    
    DataFrame(stream_error = err.stream,
              length_error = err.lengths,
              stream3 = streamprop(df3.percepts,df3.length),
              stream6 = streamprop(df6.percepts,df6.length),
              stream12 = streamprop(df12.percepts,df12.length),
              eratio = error_ratio(err,herr))
end
head(df_summary)

┌ Warning: combine(gd) is deprecated, use DataFrame(gd) instead
│   caller = |>(::GroupedDataFrame{DataFrame}, ::typeof(combine)) at operators.jl:813
└ @ Base ./operators.jl:813
┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[14]:18
└ @ Core In[14]:18


,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,f_τ_a,f_τ_m,s_τ_a,s_τ_m,t_τ_a,t_τ_m,f_W_m_σ,s_W_m_σ,t_W_m_σ_t,t_W_m_σ_ϕ,t_W_m_σ_N,stream_error,length_error,stream3,stream6,stream12,eratio
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Quantity…,Quantity…,Quantity…,Quantity…,Quantity…,Quantity…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.223436,0.232316,0.0649255,0.95678,1.0,1.15135
2,0.0,5.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.240384,0.495654,0.0488483,0.983754,1.0,1.61436
3,0.0,63.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.252312,1.0,0.0251959,1.0,1.0,2.42798
4,0.0,790.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.253495,1.0,0.0,1.0,1.0,2.43219
5,0.0,10000.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.253371,1.0,0.000759713,1.0,1.0,2.43175
6,5.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.26239,0.315528,0.00178035,0.179899,0.729756,1.41737


In [15]:
dfplot = @linq df_summary |>
    rename_levels_for([:stream_error,:length_error,:eratio,:stream3,:stream6,:stream12],suffixes=[:c_a,:c_m,:τ_a,:τ_m,:c_σ,:W_m_σ,:W_m_σ_t,:W_m_σ_ϕ,:W_m_σ_N]) |>
    transform(a = packaxes(:c_a), m = packaxes(:c_m), rand = randn(length(:c_a))) 
              
head(dfplot)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[15]:4
└ @ Core In[15]:4


,c_a,c_m,τ_a,τ_m,c_σ,W_m_σ,W_m_σ_t,W_m_σ_ϕ,W_m_σ_N,level,stream_error,length_error,eratio,stream3,stream6,stream12,a,m,rand
,Float64,Float64,Quantity…,Quantity…,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.223436,0.232316,1.15135,0.0649255,0.95678,1.0,0.0,0.0,-1.21439
2,0.0,5.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.240384,0.495654,1.61436,0.0488483,0.983754,1.0,0.0,1.5,0.0856141
3,0.0,63.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.252312,1.0,2.42798,0.0251959,1.0,1.0,0.0,2.5,-1.70686
4,0.0,790.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253495,1.0,2.43219,0.0,1.0,1.0,0.0,3.5,0.451632
5,0.0,10000.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253371,1.0,2.43175,0.000759713,1.0,1.0,0.0,5.0,-1.32757
6,5.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.26239,0.315528,1.41737,0.00178035,0.179899,0.729756,1.5,0.0,0.655045


In [16]:
dfplot[:variation] = "Aunknown"
dfplot[:varvalue] = NaN
allsel = []
sel = findall(!isequal(3.0s),dfplot.τ_a)
dfplot[sel,:variation] = "Bτ_a"
dfplot[sel,:varvalue] = ustrip.(uconvert.(s,dfplot[sel,:τ_a]))
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall(!isequal(350ms),dfplot.τ_m)
dfplot[sel,:variation] = "Cτ_m"
dfplot[sel,:varvalue] = ustrip.(uconvert.(s,dfplot[sel,:τ_m]))
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall(!isequal(0.2),dfplot.c_σ)
dfplot[sel,:variation] = "Dc_σ"
dfplot[sel,:varvalue] = dfplot[sel,:c_σ]
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall((dfplot.W_m_σ .!= 5.6) .& (dfplot.level .== "Peripheral") .& .!isnan.(dfplot.W_m_σ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ]/5.6

sel = findall((dfplot.W_m_σ .!= 15) .& (dfplot.level .== "Cortical") .& .!isnan.(dfplot.W_m_σ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ]/15

sel = findall((dfplot.W_m_σ_t .!= 7.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_t))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_t]/7.0

sel = findall((dfplot.W_m_σ_ϕ .!= 7.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_ϕ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "Zdiscard"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_ϕ]

sel = findall((dfplot.W_m_σ_N .!= 3.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_N))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ2"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_N]/3.0;

In [20]:
sel = findall((dfplot.W_m_σ .!= 5.6) .& (dfplot.level .!= "Object"))
length(sel)

350

In [17]:
first(dfplot,6)

,c_a,c_m,τ_a,τ_m,c_σ,W_m_σ,W_m_σ_t,W_m_σ_ϕ,W_m_σ_N,level,stream_error,length_error,eratio,stream3,stream6,stream12,a,m,rand,variation,varvalue
,Float64,Float64,Quantity…,Quantity…,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64
1,0.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.223436,0.232316,1.15135,0.0649255,0.95678,1.0,0.0,0.0,-1.21439,Bτ_a,1.0
2,0.0,5.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.240384,0.495654,1.61436,0.0488483,0.983754,1.0,0.0,1.5,0.0856141,Bτ_a,1.0
3,0.0,63.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.252312,1.0,2.42798,0.0251959,1.0,1.0,0.0,2.5,-1.70686,Bτ_a,1.0
4,0.0,790.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253495,1.0,2.43219,0.0,1.0,1.0,0.0,3.5,0.451632,Bτ_a,1.0
5,0.0,10000.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253371,1.0,2.43175,0.000759713,1.0,1.0,0.0,5.0,-1.32757,Bτ_a,1.0
6,5.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.26239,0.315528,1.41737,0.00178035,0.179899,0.729756,1.5,0.0,0.655045,Bτ_a,1.0


In [18]:
dfsumplot = by(dfplot,[:variation,:varvalue,:level]) do df
    lower, upper = dbootconf(shuffle(df.eratio))
    thresh = 1.35
    
    DataFrame(eratio = mean(df.eratio),
              min_eratio = minimum(df.eratio),
              min_stream_error = minimum(df.stream_error),
              min_length_error = minimum(df.length_error),
              stream3 = mean(df.stream3[df.eratio .< thresh]),
              stream6 = mean(df.stream6[df.eratio .< thresh]),
              stream12 = mean(df.stream12[df.eratio .< thresh]),
              emin = minimum(df.eratio),
              emax = maximum(df.eratio),
              elower = lower, #quantile(df.eratio,0.025),
              eupper = upper) #quantile(df.eratio,0.975))
end;

In [19]:
last(sort!(dfsumplot,:variation),25)

,variation,varvalue,level,eratio,min_eratio,min_stream_error,min_length_error,stream3,stream6,stream12,emin,emax,elower,eupper
,String,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Cτ_m,0.15,Cortical,1.70218,0.756315,0.061914,0.14457,0.108805,0.611057,0.824468,0.756315,2.8137,1.47695,1.92472
2,Cτ_m,0.5,Cortical,1.63036,0.756146,0.0617892,0.196125,0.110442,0.64018,0.874917,0.756146,2.77971,1.36833,1.89252
3,Cτ_m,1.0,Cortical,1.67634,0.917542,0.0743507,0.215,0.0786135,0.691343,0.8855,0.917542,2.77971,1.51018,1.83927
4,Cτ_m,0.15,Object,1.33967,0.65202,0.0600493,0.246539,0.0770106,0.566042,0.906787,0.65202,1.91141,1.17349,1.51286
5,Cτ_m,0.5,Object,1.17675,0.661764,0.0671892,0.226628,0.084274,0.537191,0.871755,0.661764,1.8842,1.06341,1.28565
6,Cτ_m,1.0,Object,1.1337,0.605733,0.0590004,0.219826,0.0815642,0.534502,0.90088,0.605733,1.83354,1.04139,1.22267
7,Dc_σ,0.4,Peripheral,2.08056,0.653843,0.0722452,0.148985,0.0635431,0.495387,0.90157,0.653843,2.95838,1.7351,2.40577
8,Dc_σ,0.8,Peripheral,1.98151,0.64779,0.0939312,0.200145,0.0812832,0.601878,0.92663,0.64779,2.77987,1.70296,2.28086
9,Dc_σ,1.2,Peripheral,1.90818,0.712653,0.0940793,0.237869,0.0917595,0.608843,0.90209,0.712653,2.77971,1.59689,2.19224


In [20]:
uniqueinds(x) = indexin(x,unique(x))
dfsumplot.xval = 3.2.*Float64.(uniqueinds(dfsumplot.variation))
for g in groupby(dfsumplot,:variation)
    sel = findall(isequal(first(g.variation)),dfsumplot.variation)
    dfsumplot[sel,:xval] = dfsumplot[sel,:xval] .+ (0.9.*uniqueinds(g.varvalue))
end
dfsumplot.xval .+= 0.3.*uniqueinds(dfsumplot.level)
dfsumplot.xval .-= 2.5

dfsumplot

,variation,varvalue,level,eratio,min_eratio,min_stream_error,min_length_error,stream3,stream6,stream12,emin,emax,elower,eupper,xval
,String,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Aunknown,NaN,Peripheral,2.24115,0.641092,0.070641,0.163826,0.0430683,0.473327,0.917056,0.641092,2.95683,1.99616,2.50397,1.9
2,Aunknown,NaN,Cortical,1.66738,0.920376,0.115874,0.233068,0.137188,0.636552,0.843243,0.920376,2.77971,1.42148,1.93424,2.2
3,Aunknown,NaN,Object,1.20853,0.609229,0.0619701,0.237989,0.0851797,0.560509,0.88264,0.609229,1.82454,1.04146,1.39754,2.5
4,Bτ_a,1.0,Peripheral,2.32257,0.945034,0.110688,0.224382,0.0352937,0.813913,1.0,0.945034,2.85694,2.08602,2.53841,5.1
5,Bτ_a,5.0,Peripheral,2.12073,0.51657,0.0808423,0.0937634,0.0492833,0.434474,0.880271,0.51657,2.82666,1.80711,2.4066,6.0
6,Bτ_a,10.0,Peripheral,2.11706,0.700681,0.113091,0.112151,0.111105,0.492203,0.867065,0.700681,2.92135,1.80259,2.41924,6.9
7,Bτ_a,1.0,Cortical,1.71412,0.838886,0.102004,0.27286,0.0797826,0.58383,0.843932,0.838886,2.78055,1.46878,1.97279,5.4
8,Bτ_a,5.0,Cortical,1.63242,0.87418,0.0766466,0.19914,0.177212,0.631967,0.853343,0.87418,2.80282,1.38623,1.90534,6.3
9,Bτ_a,10.0,Cortical,1.57275,0.907464,0.0825287,0.132694,0.209222,0.60949,0.875708,0.907464,2.77971,1.34637,1.79742,7.2


In [21]:
dfsumplot[occursin.(Ref(r"^O.*ϕ.*"),dfsumplot.variation),:xval]

0-element Array{Float64,1}

In [22]:
p = plot(dfsumplot,x=:xval,y=:eratio,group=:varvalue,color=:level,Geom.bar,
        Guide.yticks(ticks=[0,1,2,3]))
#draw(SVG(12inch,3inch),p)
draw(PDF(joinpath(plotdir,"fig5_raw.pdf"),12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/09PWZ/src/Gadfly.jl:484


In [38]:
p = plot(dfsumplot,x=:xval,y=:min_stream_error,group=:varvalue,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [73]:
p = plot(dfsumplot,x=:xval,y=:min_stream_error,group=:varvalue,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [39]:
mean(dfsumplot.min_stream_error), std(dfsumplot.min_stream_error)

(0.07894799426471197, 0.030965383101050174)

In [40]:
p = plot(dfsumplot,x=:xval,y=:min_length_error,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [76]:
mean(dfsumplot.min_length_error), std(dfsumplot.min_length_error)

(0.1973737602710632, 0.03124902478234511)

In [82]:
mean(dfsumplot.min_eratio), std(dfsumplot.min_eratio), maximum(dfsumplot.min_eratio)

(0.8748461794648783, 0.18955331980158496, 1.316721293120378)

In [41]:
p = plot(dfsumplot,x=:xval,y=:stream6,ymin=:stream3,ymax=:stream12,group=:varvalue,color=:level,Geom.point,Geom.errorbar)
draw(SVG(12inch,1.8inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [42]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.τ_a .!= 3.0s,:],x=:level,y=:eratio,ygroup=:τ_a,
        Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [43]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.τ_m .!= 350ms,:],x=:level,y=:eratio,ygroup=:τ_m,Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [44]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.c_σ .!= 0.2,:],x=:level,y=:eratio,ygroup=:c_σ,Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [ ]:
# TODO: visually verify the overall distribution for the inhibition parameters